<a href="https://colab.research.google.com/github/JorgePeje/test-python-utec/blob/main/Programacion-Python-Utec/proyectofinal/Intro_a_LLMs_Groq_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain-groq
#Paquete de Python
#Es un conector oficial entre LangChain y la API de Groq, una empresa que ofrece acceso muy rápido a modelos de lenguaje como LLaMA 3.

#Permite:
#Usar los modelos de Groq (como llama3-8b o llama3-70b) directamente desde LangChain
#Integrarlos fácilmente en tus cadenas de procesamiento de texto
#Hacer chatbots, asistentes, o apps que consulten tus documentos técnicos, como tu manual

#Es util porque combina lo mejor de ambos mundos
#- LangChain:	Crear flujos de procesamiento inteligente con lenguaje natural
#- Groq:	Acceder a modelos LLM potentes y ultra rápidos (como LLaMA 3)
#--> langchain-groq:	Conectar ambos fácilmente

In [1]:
'''📌 Desglose
! → Este símbolo al inicio indica que se está ejecutando un comando de terminal (shell) desde un entorno de notebook. En Python puro (por ejemplo, en scripts .py), no usarías !, pero en Jupyter sí.
pip install → Es el comando estándar de Python para instalar paquetes desde PyPI, el repositorio oficial de paquetes de Python.
langchain-groq → Es un paquete específico que integra LangChain (un framework para construir aplicaciones basadas en modelos de lenguaje) con Groq, una empresa que proporciona hardware ultrarrápido para ejecutar modelos como LLaMA, Mixtral, etc.'''

'📌 Desglose\n! → Este símbolo al inicio indica que se está ejecutando un comando de terminal (shell) desde un entorno de notebook. En Python puro (por ejemplo, en scripts .py), no usarías !, pero en Jupyter sí.\n\npip install → Es el comando estándar de Python para instalar paquetes desde PyPI, el repositorio oficial de paquetes de Python.\n\nlangchain-groq → Es un paquete específico que integra LangChain (un framework para construir aplicaciones basadas en modelos de lenguaje) con Groq, una empresa que proporciona hardware ultrarrápido para ejecutar modelos como LLaMA, Mixtral, etc.'

In [ ]:
import os
#from dotenv import load_dotenv
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_groq import ChatGroq

In [ ]:
#VISION_MODEL_NAME="llama-3.2-11b-vision-preview"

In [ ]:
#Aca se esta generando la conexion a LLM.

# ChatGroq es una clase
#Temperatura= 0 (no seas tan verboso, se directo npo agregues mucho texto) // 1: Agrega mas detalle, mas diversidad (sinonimos y antonimos), pero puede alucinar

llm = ChatGroq(
    temperature=0,
    model_name="llama3-8b-8192", # modelo de llama utilizado
    groq_api_key="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" #API Groq descargada
)

In [ ]:
#Instruccion o PROMPT (lo que queremos que haga).
# Deberiamos poner: "Dada cierto informacion (manual) quiero que me respondas la siguiente pregunta"

#Template:  variable con texto
# {recibo} : es una variable
# {format_instructions} : Parser --> Indica a la LLM como se espera el output que generara este

TEMPLATE="""
Dado la informacion del siguiente recibo, necesito que me extraigas la empresa emisora y el tipo de servicio
{recibo}

El formato de salida debe ser el siguiente:{format_instructions}
"""

In [ ]:
# POO para el parser formato de salida --> Abstraen procesos de la vida real es la Clase:  Plantilla que se asemeja a la realidad

class ContentGenerationScript(BaseModel):
#Se indica que se espera de salida de la LLM, que quiero que me devuelva:
    empresa_emisora: str = Field(description="Nombre de la empresa emisora")
    tipo_servicio: str = Field(description="Nombre del tipo de servicio")

#JsonOutputParser es otra clase, quiero que me devuelva la respuesta en un diccionario.
# Todo se devuelve dentro de la variable "parser"
parser= JsonOutputParser(pydantic_object=ContentGenerationScript)

In [ ]:





prompt = PromptTemplate(
    input_variables=["topic"],
    template=TEMPLATE,
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
# parser variable .get_format-...

chain = LLMChain(llm=llm, prompt=prompt, output_parser=parser)
#Objeto que permite concatenar todo y permite ejecturar todo el proceso


recibo="""
Nº: FAC-2024-5582
Fecha: 20 de mayo de 2024

EMISOR:
• Razón Social: TecnoSoluciones Innovadoras S.A. de C.V.
• RFC: TSI-850301-KL4
• Dirección: Calle Tecnológico #742, Guadalajara, Jalisco.
• Contacto: ventas@tecnosoluciones.com | Tel. 33-5678-9012

CLIENTE: Roberto Sánchez García
CANTIDAD RECIBIDA: $3,800.00 MXN *(Tres mil ochocientos pesos 00/100 MXN)*

🛠 SERVICIO PRESTADO:
• Tipo de servicio: Mantenimiento preventivo de equipo de cómputo (5 laptops y 3 impresoras).
• Descripción detallada:

Limpieza física de hardware.

Actualización de software y antivirus.

Revisión de componentes eléctricos.

FORMA DE PAGO: Tarjeta de crédito (VISA terminación 7890)
REFERENCIA: *OP-TSI-2024-0055*

IMPORTE DESGLOSADO:

Subtotal: $3,275.86 MXN

IVA (16%): $524.14 MXN

Total: $3,800.00 MXN

FIRMA Y SELLO:

Atentamente,
Lic. Fernanda López
Gerente de Servicios Técnicos
"TecnoSoluciones Innovadoras"

Nota: *Este documento es válido como comprobante fiscal (Art. 29-A del CFF).*

"""


result = chain.invoke({"recibo": recibo})
# Aca estoy ejecutando la cadena y se le pasa la variable de entrada, "recibo"


result

{'recibo': '\nNº: FAC-2024-5582\nFecha: 20 de mayo de 2024\n\nEMISOR:\n• Razón Social: TecnoSoluciones Innovadoras S.A. de C.V.\n• RFC: TSI-850301-KL4\n• Dirección: Calle Tecnológico #742, Guadalajara, Jalisco.\n• Contacto: ventas@tecnosoluciones.com | Tel. 33-5678-9012\n\nCLIENTE: Roberto Sánchez García\nCANTIDAD RECIBIDA: $3,800.00 MXN *(Tres mil ochocientos pesos 00/100 MXN)*\n\n🛠 SERVICIO PRESTADO:\n• Tipo de servicio: Mantenimiento preventivo de equipo de cómputo (5 laptops y 3 impresoras).\n• Descripción detallada:\n\nLimpieza física de hardware.\n\nActualización de software y antivirus.\n\nRevisión de componentes eléctricos.\n\nFORMA DE PAGO: Tarjeta de crédito (VISA terminación 7890)\nREFERENCIA: *OP-TSI-2024-0055*\n\nIMPORTE DESGLOSADO:\n\nSubtotal: $3,275.86 MXN\n\nIVA (16%): $524.14 MXN\n\nTotal: $3,800.00 MXN\n\nFIRMA Y SELLO:\n\nAtentamente,\nLic. Fernanda López\nGerente de Servicios Técnicos\n"TecnoSoluciones Innovadoras"\n\nNota: *Este documento es válido como comprobant

In [ ]:
result['text']

{'empresa_emisora': 'TecnoSoluciones Innovadoras S.A. de C.V.',
 'tipo_servicio': 'Mantenimiento preventivo de equipo de cómputo'}

In [ ]:
# tarea con el manual del camion:
# Generar instruccion: dado cierto contexto (dar el contexto-->parseo del pdf, pedazo del manual) quiero que me respondas la siguiente pregunta (siendo la pregunta, una variable)
# tanto la pregunta y el contexto tienen que ser dinamicos

#Contexto : Pedazo de texto del manual. (Pedazo del manual)
#Pregunta: Pregunta que haga el usuario.

#Estas dos dinamicos, siempre va recibir como varible la instruccion (prompt)

#Cambiar la instruccion, cambiar "recibo" por "contexto" // agregar la variabe "pregunta" y sacar un pedazo de texto de lo parseado.
